In [ ]:
#Importing libraries
import requests
from bs4 import BeautifulSoup as bs
from requests import get
import pandas as pd
from datetime import date
from datetime import datetime
import time

In [ ]:
#create timestamp from string in "mm/dd/yyyy" format
def maketimestamp(s):
    #convert string into datetime format
    date = datetime.strptime(s, "%m/%d/%Y")

    #raise exception if date is outside range
    today = date.today()
    startdate = datetime.strptime("07/01/2010", "%m/%d/%Y")
    if date < startdate or date > today:
        raise Exception("Date must be between July 1, 2010, and Today")
    
    #convert datetime to timestamp and return
    timestamp = time.mktime(date.timetuple())
    return(int(timestamp))

#create URL for incident search query from two dates
def makeurl(start, end):
    #if start is empty or set to "start", use starting date for query
    if start == "start":
        startdate = 1277960400
    else:
        startdate = maketimestamp(start)

    #if end is empty or set to "today", use today for query
    if end == "today":
        enddate = int(time.mktime(date.today().timetuple()))
    else:
        enddate = maketimestamp(end)
    
    #create url from start and end dates and return
    url = "https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=" \
        + str(startdate) + "&endDate=" + str(enddate)
    return(url)

#find number of pages generated by a query
def findpagenum(url):
    #find div with page number class
    page = get(url)
    soup = bs(page.content, 'html.parser')
    mydivs = soup.find_all("li", {"class": "page-count"})
    pagecount = str(mydivs[0].findChildren('span'))

    #clean html string and return page count
    pagecount = pagecount.split("/")[1].split("<")[0]
    return(int(pagecount))
    
#scrape website for all reports given an URL
def read(url):
    #iterate through all pages of incident query
    for i in range(0, findpagenum(url)):
        #change URL for each page
        offset = i * 5
        webpage =  url + str(offset)
        
        #create dataframe for first page
        if offset == 0:
            df = pd.read_html(webpage)[0]

        #apend data from new pages to dataframe
        else:
            #get table data
            try:
                newdata = pd.read_html(webpage)[0]
                df = df.append(newdata)
            except:
                print("Error! Broken Page: " + website)
    
    #reset index and return datafram
    df = df.reset_index().drop(["index"], axis=1)
    return(df)

#scrapes UChicago Incidents Page between two dates
#primary arguement is end date, secondary arguement is starting date
#default is starting date to today
#dates must be in "mm/dd/yyyy" format
def scrape(end = "today", start = "start"):
    url = makeurl(start, end)
    df = read(url)
    return(df)

In [ ]:
#iterates through pages, and creates website link string
df = scrape("08/01/2010")

In [ ]:
df.to_csv("newoutput.csv")